In [1]:
sc
from pyspark.sql import SQLContext
from pyspark.sql import Row

In [2]:
flightsPath = "hdfs://localhost:9000/flights.csv"

In [3]:
flightsData = sc.textFile(flightsPath)

In [4]:
flightsData.first()

u'"FL_DATE","AIRLINE_ID","FL_NUM","ORIGIN","DEST","DEP_TIME","DEP_DELAY","ARR_TIME","ARR_DELAY","AIR_TIME","DISTANCE",'

In [5]:
#  notHeader()    function to remove Header


def notHeader(row):
    return "AIRLINE_ID" not in row

In [94]:
# split()

import csv 
from StringIO import StringIO

def split(line):
    reader = csv.reader(StringIO(line))
    return reader.next()

In [95]:
d1 = flightsData.filter(notHeader)
d1.take(3)

[u'2016-01-01,19790,"1248","DTW","LAX","1935",0.00,"2120",-24.00,249.00,1979.00,',
 u'2016-01-01,19790,"1251","ATL","GRR","2130",5.00,"2319",-2.00,92.00,640.00,',
 u'2016-01-01,19790,"1254","LAX","ATL","2256",1.00,"0547",-13.00,207.00,1947.00,']

In [97]:
d2 = d1.map(split)
d2.take(2)

[['2016-01-01',
  '19790',
  '1248',
  'DTW',
  'LAX',
  '1935',
  '0.00',
  '2120',
  '-24.00',
  '249.00',
  '1979.00',
  ''],
 ['2016-01-01',
  '19790',
  '1251',
  'ATL',
  'GRR',
  '2130',
  '5.00',
  '2319',
  '-2.00',
  '92.00',
  '640.00',
  '']]

In [99]:
d2.take(241)

[['2016-01-01',
  '19790',
  '1248',
  'DTW',
  'LAX',
  '1935',
  '0.00',
  '2120',
  '-24.00',
  '249.00',
  '1979.00',
  ''],
 ['2016-01-01',
  '19790',
  '1251',
  'ATL',
  'GRR',
  '2130',
  '5.00',
  '2319',
  '-2.00',
  '92.00',
  '640.00',
  ''],
 ['2016-01-01',
  '19790',
  '1254',
  'LAX',
  'ATL',
  '2256',
  '1.00',
  '0547',
  '-13.00',
  '207.00',
  '1947.00',
  ''],
 ['2016-01-01',
  '19790',
  '1255',
  'SLC',
  'ATL',
  '1700',
  '4.00',
  '2213',
  '-16.00',
  '173.00',
  '1590.00',
  ''],
 ['2016-01-01',
  '19790',
  '1256',
  'BZN',
  'MSP',
  '1012',
  '72.00',
  '1420',
  '124.00',
  '121.00',
  '874.00',
  ''],
 ['2016-01-01',
  '19790',
  '1257',
  'ATL',
  'BNA',
  '1356',
  '83.00',
  '1402',
  '83.00',
  '38.00',
  '214.00',
  ''],
 ['2016-01-01',
  '19790',
  '1257',
  'BNA',
  'ATL',
  '1446',
  '86.00',
  '1644',
  '74.00',
  '37.00',
  '214.00',
  ''],
 ['2016-01-01',
  '19790',
  '1258',
  'ATL',
  'JAX',
  '0946',
  '1.00',
  '1053',
  '3.00',
  '45.00'

###### using Row object

In [104]:
from datetime import datetime 

d3 = d2.map(lambda x: Row(date= datetime.strptime(x[0], "%Y-%m-%d"), airline = x[1], flightnum=x[2], origin=x[3], dest=x[4], dep=x[5],
                                                              dep_delay=x[6], arv=x[7], arv_delay=x[8],
                                                               airtime=x[9], distance=float(x[10]) ))

In [105]:
d3.take(5)

[Row(airline='19790', airtime='249.00', arv='2120', arv_delay='-24.00', date=datetime.datetime(2016, 1, 1, 0, 0), dep='1935', dep_delay='0.00', dest='LAX', distance=1979.0, flightnum='1248', origin='DTW'),
 Row(airline='19790', airtime='92.00', arv='2319', arv_delay='-2.00', date=datetime.datetime(2016, 1, 1, 0, 0), dep='2130', dep_delay='5.00', dest='GRR', distance=640.0, flightnum='1251', origin='ATL'),
 Row(airline='19790', airtime='207.00', arv='0547', arv_delay='-13.00', date=datetime.datetime(2016, 1, 1, 0, 0), dep='2256', dep_delay='1.00', dest='ATL', distance=1947.0, flightnum='1254', origin='LAX'),
 Row(airline='19790', airtime='173.00', arv='2213', arv_delay='-16.00', date=datetime.datetime(2016, 1, 1, 0, 0), dep='1700', dep_delay='4.00', dest='ATL', distance=1590.0, flightnum='1255', origin='SLC'),
 Row(airline='19790', airtime='121.00', arv='1420', arv_delay='124.00', date=datetime.datetime(2016, 1, 1, 0, 0), dep='1012', dep_delay='72.00', dest='MSP', distance=874.0, flight

In [106]:
d3.\
    filter(lambda x:x['date'] == datetime.strptime("2016-01-01", "%Y-%m-%d") ).\
    count()

13019

In [107]:
d3.\
    filter(lambda x:x['distance']== 1590.0).\
    take(1)

[Row(airline='19790', airtime='173.00', arv='2213', arv_delay='-16.00', date=datetime.datetime(2016, 1, 1, 0, 0), dep='1700', dep_delay='4.00', dest='ATL', distance=1590.0, flightnum='1255', origin='SLC')]

In [15]:
d3.\
    filter(lambda x:x['distance'] >= 2000).\
    take(1)

[Row(airline='19790', airtime='279.00', arv='0701', arv_delay='-14.00', date=datetime.datetime(2016, 1, 1, 0, 0), dep='2248', dep_delay='-7.00', dest='JFK', distance=2475.0, flightnum='1262', origin='LAX')]

In [16]:
d3.\
    filter(lambda x:x['distance'] >= 3000).\
    count()

806

In [17]:
d3.\
    filter(lambda x:x['distance'] >= 5000).\
    count()

0

In [18]:
d3.\
    filter(lambda x:x['dest']== "LAX").\
    take(1)

[Row(airline='19790', airtime='249.00', arv='2120', arv_delay='-24.00', date=datetime.datetime(2016, 1, 1, 0, 0), dep='1935', dep_delay='0.00', dest='LAX', distance=1979.0, flightnum='1248', origin='DTW')]

In [21]:
d3.\
    filter(lambda x:(x['airline']== "19790") and (x['date'] == datetime.strptime("2016-01-01", "%Y-%m-%d")) and (( x['dest'] == 'LAX' or x['dest'] == 'ATL' or x['dest'] == 'MSP' ))).\
    take(50)

[Row(airline='19790', airtime='249.00', arv='2120', arv_delay='-24.00', date=datetime.datetime(2016, 1, 1, 0, 0), dep='1935', dep_delay='0.00', dest='LAX', distance=1979.0, flightnum='1248', origin='DTW'),
 Row(airline='19790', airtime='207.00', arv='0547', arv_delay='-13.00', date=datetime.datetime(2016, 1, 1, 0, 0), dep='2256', dep_delay='1.00', dest='ATL', distance=1947.0, flightnum='1254', origin='LAX'),
 Row(airline='19790', airtime='173.00', arv='2213', arv_delay='-16.00', date=datetime.datetime(2016, 1, 1, 0, 0), dep='1700', dep_delay='4.00', dest='ATL', distance=1590.0, flightnum='1255', origin='SLC'),
 Row(airline='19790', airtime='121.00', arv='1420', arv_delay='124.00', date=datetime.datetime(2016, 1, 1, 0, 0), dep='1012', dep_delay='72.00', dest='MSP', distance=874.0, flightnum='1256', origin='BZN'),
 Row(airline='19790', airtime='37.00', arv='1644', arv_delay='74.00', date=datetime.datetime(2016, 1, 1, 0, 0), dep='1446', dep_delay='86.00', dest='ATL', distance=214.0, fligh

In [23]:
d3.\
    filter(lambda x:x['dest']== "LAX").\
    filter(lambda x:x['airline']== "19790").\
    count()

2623

In [24]:
d3.\
    filter(lambda x: (x['dest']== "LAX")  and (x['airline']== "19790") ).\
    count()

2623

# Part 2
#### using class and namedtuple


In [1]:
sc

In [2]:
flightsPath = "hdfs://localhost:9000/flights.csv"

In [3]:
flightsData = sc.textFile(flightsPath)

In [4]:
flightsData.first()

u'"FL_DATE","AIRLINE_ID","FL_NUM","ORIGIN","DEST","DEP_TIME","DEP_DELAY","ARR_TIME","ARR_DELAY","AIR_TIME","DISTANCE",'

In [5]:
flightsData.take(717)   # check below some data is blank for time data, distance data
                        # also notice that there is comma at the end of each record in data

[u'"FL_DATE","AIRLINE_ID","FL_NUM","ORIGIN","DEST","DEP_TIME","DEP_DELAY","ARR_TIME","ARR_DELAY","AIR_TIME","DISTANCE",',
 u'2016-01-01,19790,"1248","DTW","LAX","1935",0.00,"2120",-24.00,249.00,1979.00,',
 u'2016-01-01,19790,"1251","ATL","GRR","2130",5.00,"2319",-2.00,92.00,640.00,',
 u'2016-01-01,19790,"1254","LAX","ATL","2256",1.00,"0547",-13.00,207.00,1947.00,',
 u'2016-01-01,19790,"1255","SLC","ATL","1700",4.00,"2213",-16.00,173.00,1590.00,',
 u'2016-01-01,19790,"1256","BZN","MSP","1012",72.00,"1420",124.00,121.00,874.00,',
 u'2016-01-01,19790,"1257","ATL","BNA","1356",83.00,"1402",83.00,38.00,214.00,',
 u'2016-01-01,19790,"1257","BNA","ATL","1446",86.00,"1644",74.00,37.00,214.00,',
 u'2016-01-01,19790,"1258","ATL","JAX","0946",1.00,"1053",3.00,45.00,270.00,',
 u'2016-01-01,19790,"1258","JAX","ATL","1144",-1.00,"1247",-15.00,43.00,270.00,',
 u'2016-01-01,19790,"1259","ATL","OKC","2107",-3.00,"2224",-12.00,116.00,761.00,',
 u'2016-01-01,19790,"1260","MSP","SMF","1113",-2.00,"1236",-

In [6]:
# join blank after comma with 00.00 
# blanks will be replaced by 00.00 along with additional 00.00 entry for each record at the end because of end comma

blanks = flightsData.map(lambda x:','.join(x or '00.00' for x in x.split(',')))

In [7]:
blanks.take(717)  # Notice that time data is still blank in some records

[u'"FL_DATE","AIRLINE_ID","FL_NUM","ORIGIN","DEST","DEP_TIME","DEP_DELAY","ARR_TIME","ARR_DELAY","AIR_TIME","DISTANCE",00.00',
 u'2016-01-01,19790,"1248","DTW","LAX","1935",0.00,"2120",-24.00,249.00,1979.00,00.00',
 u'2016-01-01,19790,"1251","ATL","GRR","2130",5.00,"2319",-2.00,92.00,640.00,00.00',
 u'2016-01-01,19790,"1254","LAX","ATL","2256",1.00,"0547",-13.00,207.00,1947.00,00.00',
 u'2016-01-01,19790,"1255","SLC","ATL","1700",4.00,"2213",-16.00,173.00,1590.00,00.00',
 u'2016-01-01,19790,"1256","BZN","MSP","1012",72.00,"1420",124.00,121.00,874.00,00.00',
 u'2016-01-01,19790,"1257","ATL","BNA","1356",83.00,"1402",83.00,38.00,214.00,00.00',
 u'2016-01-01,19790,"1257","BNA","ATL","1446",86.00,"1644",74.00,37.00,214.00,00.00',
 u'2016-01-01,19790,"1258","ATL","JAX","0946",1.00,"1053",3.00,45.00,270.00,00.00',
 u'2016-01-01,19790,"1258","JAX","ATL","1144",-1.00,"1247",-15.00,43.00,270.00,00.00',
 u'2016-01-01,19790,"1259","ATL","OKC","2107",-3.00,"2224",-12.00,116.00,761.00,00.00',
 u'20

In [8]:
# replacing blank time data with 0000 in required format as "0000"

blanktime = blanks.map(lambda x:x.replace(',""' , ',"0000"') )

In [9]:
finalF = blanktime                

In [10]:
finalF.take(242)

[u'"FL_DATE","AIRLINE_ID","FL_NUM","ORIGIN","DEST","DEP_TIME","DEP_DELAY","ARR_TIME","ARR_DELAY","AIR_TIME","DISTANCE",00.00',
 u'2016-01-01,19790,"1248","DTW","LAX","1935",0.00,"2120",-24.00,249.00,1979.00,00.00',
 u'2016-01-01,19790,"1251","ATL","GRR","2130",5.00,"2319",-2.00,92.00,640.00,00.00',
 u'2016-01-01,19790,"1254","LAX","ATL","2256",1.00,"0547",-13.00,207.00,1947.00,00.00',
 u'2016-01-01,19790,"1255","SLC","ATL","1700",4.00,"2213",-16.00,173.00,1590.00,00.00',
 u'2016-01-01,19790,"1256","BZN","MSP","1012",72.00,"1420",124.00,121.00,874.00,00.00',
 u'2016-01-01,19790,"1257","ATL","BNA","1356",83.00,"1402",83.00,38.00,214.00,00.00',
 u'2016-01-01,19790,"1257","BNA","ATL","1446",86.00,"1644",74.00,37.00,214.00,00.00',
 u'2016-01-01,19790,"1258","ATL","JAX","0946",1.00,"1053",3.00,45.00,270.00,00.00',
 u'2016-01-01,19790,"1258","JAX","ATL","1144",-1.00,"1247",-15.00,43.00,270.00,00.00',
 u'2016-01-01,19790,"1259","ATL","OKC","2107",-3.00,"2224",-12.00,116.00,761.00,00.00',
 u'20

In [11]:
finalF.take(701) # here all blank data is replaced with 00.00 for float data and "0000" for time data

[u'"FL_DATE","AIRLINE_ID","FL_NUM","ORIGIN","DEST","DEP_TIME","DEP_DELAY","ARR_TIME","ARR_DELAY","AIR_TIME","DISTANCE",00.00',
 u'2016-01-01,19790,"1248","DTW","LAX","1935",0.00,"2120",-24.00,249.00,1979.00,00.00',
 u'2016-01-01,19790,"1251","ATL","GRR","2130",5.00,"2319",-2.00,92.00,640.00,00.00',
 u'2016-01-01,19790,"1254","LAX","ATL","2256",1.00,"0547",-13.00,207.00,1947.00,00.00',
 u'2016-01-01,19790,"1255","SLC","ATL","1700",4.00,"2213",-16.00,173.00,1590.00,00.00',
 u'2016-01-01,19790,"1256","BZN","MSP","1012",72.00,"1420",124.00,121.00,874.00,00.00',
 u'2016-01-01,19790,"1257","ATL","BNA","1356",83.00,"1402",83.00,38.00,214.00,00.00',
 u'2016-01-01,19790,"1257","BNA","ATL","1446",86.00,"1644",74.00,37.00,214.00,00.00',
 u'2016-01-01,19790,"1258","ATL","JAX","0946",1.00,"1053",3.00,45.00,270.00,00.00',
 u'2016-01-01,19790,"1258","JAX","ATL","1144",-1.00,"1247",-15.00,43.00,270.00,00.00',
 u'2016-01-01,19790,"1259","ATL","OKC","2107",-3.00,"2224",-12.00,116.00,761.00,00.00',
 u'20

In [12]:
def notHeader(row):
    return "AIRLINE_ID" not in row

In [13]:
import csv 
from StringIO import StringIO

def split(line):
    reader = csv.reader(StringIO(line))
    return reader.next()

In [14]:

from datetime import datetime 
from collections import namedtuple

fields   = ('date', 'airline', 'flightnum', 'origin', 'dest', 'dep',
            'dep_delay', 'arv', 'arv_delay', 'airtime', 'distance')

Flight   = namedtuple('Flight', fields, verbose=False)

DATE_FMT = '%Y-%m-%d'

TIME_FMT = '%H%M%S' # %S is added to remove error "unconverted data remains 0 " in strptime()


def parse(row):
    row[0]  = datetime.strptime(row[0], DATE_FMT).date()
    row[5]  = datetime.strptime(row[5], TIME_FMT).time()
    row[6]  = float(row[6])
    row[7]  = datetime.strptime(row[7], TIME_FMT).time()
    row[8]  = float(row[8])
    row[9]  = float(row[9])
    row[10] = float(row[10])
    return Flight(*row[:11])

In [15]:
f = finalF.filter(notHeader).map(split)

In [16]:
f.take(717)

[['2016-01-01',
  '19790',
  '1248',
  'DTW',
  'LAX',
  '1935',
  '0.00',
  '2120',
  '-24.00',
  '249.00',
  '1979.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1251',
  'ATL',
  'GRR',
  '2130',
  '5.00',
  '2319',
  '-2.00',
  '92.00',
  '640.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1254',
  'LAX',
  'ATL',
  '2256',
  '1.00',
  '0547',
  '-13.00',
  '207.00',
  '1947.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1255',
  'SLC',
  'ATL',
  '1700',
  '4.00',
  '2213',
  '-16.00',
  '173.00',
  '1590.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1256',
  'BZN',
  'MSP',
  '1012',
  '72.00',
  '1420',
  '124.00',
  '121.00',
  '874.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1257',
  'ATL',
  'BNA',
  '1356',
  '83.00',
  '1402',
  '83.00',
  '38.00',
  '214.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1257',
  'BNA',
  'ATL',
  '1446',
  '86.00',
  '1644',
  '74.00',
  '37.00',
  '214.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1258',
  'ATL',
  'JAX',
  '0946',
  '1

In [17]:
f.take(15416)

[['2016-01-01',
  '19790',
  '1248',
  'DTW',
  'LAX',
  '1935',
  '0.00',
  '2120',
  '-24.00',
  '249.00',
  '1979.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1251',
  'ATL',
  'GRR',
  '2130',
  '5.00',
  '2319',
  '-2.00',
  '92.00',
  '640.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1254',
  'LAX',
  'ATL',
  '2256',
  '1.00',
  '0547',
  '-13.00',
  '207.00',
  '1947.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1255',
  'SLC',
  'ATL',
  '1700',
  '4.00',
  '2213',
  '-16.00',
  '173.00',
  '1590.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1256',
  'BZN',
  'MSP',
  '1012',
  '72.00',
  '1420',
  '124.00',
  '121.00',
  '874.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1257',
  'ATL',
  'BNA',
  '1356',
  '83.00',
  '1402',
  '83.00',
  '38.00',
  '214.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1257',
  'BNA',
  'ATL',
  '1446',
  '86.00',
  '1644',
  '74.00',
  '37.00',
  '214.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1258',
  'ATL',
  'JAX',
  '0946',
  '1

In [18]:
fp = f.map(parse)

In [19]:
fp.take(1)

[Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1248', origin='DTW', dest='LAX', dep=datetime.time(19, 3, 5), dep_delay=0.0, arv=datetime.time(21, 2), arv_delay=-24.0, airtime=249.0, distance=1979.0)]

In [20]:
fp.take(15000)

[Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1248', origin='DTW', dest='LAX', dep=datetime.time(19, 3, 5), dep_delay=0.0, arv=datetime.time(21, 2), arv_delay=-24.0, airtime=249.0, distance=1979.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1251', origin='ATL', dest='GRR', dep=datetime.time(21, 3), dep_delay=5.0, arv=datetime.time(23, 1, 9), arv_delay=-2.0, airtime=92.0, distance=640.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1254', origin='LAX', dest='ATL', dep=datetime.time(22, 5, 6), dep_delay=1.0, arv=datetime.time(5, 4, 7), arv_delay=-13.0, airtime=207.0, distance=1947.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1255', origin='SLC', dest='ATL', dep=datetime.time(17, 0), dep_delay=4.0, arv=datetime.time(22, 1, 3), arv_delay=-16.0, airtime=173.0, distance=1590.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1256', origin='BZN', dest='MSP', dep=datetime.tim

In [71]:
date_condition= fp.filter((lambda x:x[0] == datetime.strptime("2016-01-01", "%Y-%m-%d").date()))

In [75]:
date_condition.take(50)

[Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1248', origin='DTW', dest='LAX', dep=datetime.time(19, 3, 5), dep_delay=0.0, arv=datetime.time(21, 2), arv_delay=-24.0, airtime=249.0, distance=1979.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1251', origin='ATL', dest='GRR', dep=datetime.time(21, 3), dep_delay=5.0, arv=datetime.time(23, 1, 9), arv_delay=-2.0, airtime=92.0, distance=640.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1254', origin='LAX', dest='ATL', dep=datetime.time(22, 5, 6), dep_delay=1.0, arv=datetime.time(5, 4, 7), arv_delay=-13.0, airtime=207.0, distance=1947.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1255', origin='SLC', dest='ATL', dep=datetime.time(17, 0), dep_delay=4.0, arv=datetime.time(22, 1, 3), arv_delay=-16.0, airtime=173.0, distance=1590.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1256', origin='BZN', dest='MSP', dep=datetime.tim

In [65]:
air_delay_condition = fp.filter(lambda x:x[8] < 0)

In [66]:
air_delay_condition.take(10)

[Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1248', origin='DTW', dest='LAX', dep=datetime.time(19, 3, 5), dep_delay=0.0, arv=datetime.time(21, 2), arv_delay=-24.0, airtime=249.0, distance=1979.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1251', origin='ATL', dest='GRR', dep=datetime.time(21, 3), dep_delay=5.0, arv=datetime.time(23, 1, 9), arv_delay=-2.0, airtime=92.0, distance=640.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1254', origin='LAX', dest='ATL', dep=datetime.time(22, 5, 6), dep_delay=1.0, arv=datetime.time(5, 4, 7), arv_delay=-13.0, airtime=207.0, distance=1947.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1255', origin='SLC', dest='ATL', dep=datetime.time(17, 0), dep_delay=4.0, arv=datetime.time(22, 1, 3), arv_delay=-16.0, airtime=173.0, distance=1590.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1258', origin='JAX', dest='ATL', dep=datetime.tim

In [73]:
arv_condition = fp.filter(lambda x:x[7] <= datetime.strptime("2120" , "%H%M%S").time() )

In [74]:
arv_condition.take(2)

[Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1248', origin='DTW', dest='LAX', dep=datetime.time(19, 3, 5), dep_delay=0.0, arv=datetime.time(21, 2), arv_delay=-24.0, airtime=249.0, distance=1979.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1254', origin='LAX', dest='ATL', dep=datetime.time(22, 5, 6), dep_delay=1.0, arv=datetime.time(5, 4, 7), arv_delay=-13.0, airtime=207.0, distance=1947.0)]

In [23]:
distance_condition = fp.filter(lambda x:x[10] == 1979.0)

In [24]:
distance_condition.take(5) 

[Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1248', origin='DTW', dest='LAX', dep=datetime.time(19, 3, 5), dep_delay=0.0, arv=datetime.time(21, 2), arv_delay=-24.0, airtime=249.0, distance=1979.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1406', origin='LAX', dest='DTW', dep=datetime.time(23, 4), dep_delay=10.0, arv=datetime.time(6, 4, 7), arv_delay=-4.0, airtime=227.0, distance=1979.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1506', origin='LAX', dest='DTW', dep=datetime.time(6, 5, 5), dep_delay=-5.0, arv=datetime.time(14, 0, 2), arv_delay=-23.0, airtime=228.0, distance=1979.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1706', origin='DTW', dest='LAX', dep=datetime.time(8, 2, 3), dep_delay=-7.0, arv=datetime.time(10, 1, 9), arv_delay=-27.0, airtime=265.0, distance=1979.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1719', origin='DTW', dest='LAX', dep=dateti

In [26]:
distance_condition.take(10) 

[Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1248', origin='DTW', dest='LAX', dep=datetime.time(19, 3, 5), dep_delay=0.0, arv=datetime.time(21, 2), arv_delay=-24.0, airtime=249.0, distance=1979.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1406', origin='LAX', dest='DTW', dep=datetime.time(23, 4), dep_delay=10.0, arv=datetime.time(6, 4, 7), arv_delay=-4.0, airtime=227.0, distance=1979.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1506', origin='LAX', dest='DTW', dep=datetime.time(6, 5, 5), dep_delay=-5.0, arv=datetime.time(14, 0, 2), arv_delay=-23.0, airtime=228.0, distance=1979.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1706', origin='DTW', dest='LAX', dep=datetime.time(8, 2, 3), dep_delay=-7.0, arv=datetime.time(10, 1, 9), arv_delay=-27.0, airtime=265.0, distance=1979.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1719', origin='DTW', dest='LAX', dep=dateti

In [63]:
date_condition = fp.filter(lambda x:x[0] == datetime.strptime("2016-01-01", "%Y-%m-%d"))

In [61]:
#date_condition = fp.filter(lambda x:x[0] == datetime.date(2016, 1, 1))

In [29]:
totalDistance=fp.map(lambda x:x.distance).reduce(lambda x,y:x+y)

In [31]:
avgDistance=totalDistance/fp.count()

In [32]:
avgDistance

844.2338799579209

In [33]:
# Find the % of flights with delays 
fp.filter(lambda x:x.dep_delay>0).count()/float(fp.count())

0.33452662131275135

In [34]:
sumCount=fp.map(lambda x:x.dep_delay).aggregate((0,0),
                                                          (lambda acc,value: (acc[0]+value, acc[1]+1)),
                                                          (lambda acc1,acc2:(acc1[0]+acc2[0],acc1[1]+acc2[1])))

In [35]:
print "The average delay is "+str(sumCount[0]/float(sumCount[1]))

The average delay is 7.59297664789


In [36]:
fp.map(lambda x:int(x.dep_delay/60)).countByValue()

defaultdict(int,
            {0: 424116,
             1: 14101,
             2: 4418,
             3: 1669,
             4: 686,
             5: 327,
             6: 150,
             7: 88,
             8: 65,
             9: 42,
             10: 33,
             11: 23,
             12: 17,
             13: 15,
             14: 17,
             15: 19,
             16: 5,
             17: 7,
             18: 10,
             19: 5,
             20: 3,
             21: 4,
             22: 2,
             23: 1,
             24: 1,
             25: 1,
             26: 1,
             27: 1})

In [37]:
airportDelays = fp.map(lambda x: (x.origin,x.dep_delay))

In [38]:
airportDelays.keys().take(10)

['DTW', 'ATL', 'LAX', 'SLC', 'BZN', 'ATL', 'BNA', 'ATL', 'JAX', 'ATL']

In [39]:
airportDelays.values().take(10)

[0.0, 5.0, 1.0, 4.0, 72.0, 83.0, 86.0, 1.0, -1.0, -3.0]

In [40]:
# First find the total delay per airport
airportTotalDelay=airportDelays.reduceByKey(lambda x,y:x+y)

# Find the count per airport
airportCount=airportDelays.mapValues(lambda x:1).reduceByKey(lambda x,y:x+y)

# Join to have the sum, count in 1 RDD 
airportSumCount=airportTotalDelay.join(airportCount)

# Compute avg delay per airport 
airportAvgDelay=airportSumCount.mapValues(lambda x : x[0]/float(x[1]))

In [41]:
airportAvgDelay.sortBy(lambda x:-x[1]).take(10)

[('COD', 53.0),
 ('MQT', 43.51851851851852),
 ('BRD', 35.80769230769231),
 ('HIB', 32.903614457831324),
 ('APN', 30.903846153846153),
 ('CMX', 29.193548387096776),
 ('OTH', 21.88235294117647),
 ('FAR', 19.89572192513369),
 ('SWF', 19.71641791044776),
 ('VLD', 18.654761904761905)]

In [ ]:
#airportsPath = "hdfs://localhost:9000/flights.csv"

In [49]:
airportsPath = "file:///home/utpal/airports.csv"

In [50]:
airports=sc.textFile(airportsPath)

In [51]:
airports.take(5)

[u'Code,Description',
 u'"01A","Afognak Lake, AK: Afognak Lake Airport"',
 u'"03A","Granite Mountain, AK: Bear Creek Mining Strip"',
 u'"04A","Lik, AK: Lik Mining Camp"',
 u'"05A","Little Squaw, AK: Little Squaw Airport"']

In [52]:
airports=sc.textFile(airportsPath).filter(notHeader).map(split)

In [53]:
airports.lookup('PPG')

['Pago Pago, TT: Pago Pago International']

In [54]:
airportLookup=airports.collectAsMap()

In [55]:
airportLookup

{'SPY': "San Pedro, Cote d'Ivoire: San Pedro Airport",
 'SPZ': 'Springdale, AR: Springdale Municipal',
 'SPP': 'Menongue, Angola: Menongue Airport',
 'SPQ': 'San Pedro, CA: Catalina Air-Sea Terminal Heliport',
 'SPR': 'San Pedro, Belize: San Pedro Airport',
 'SPS': 'Wichita Falls, TX: Sheppard AFB/Wichita Falls Municipal',
 'SPU': 'Split, Croatia: Split Airport',
 'SPW': 'Spencer, IA: Spencer Municipal',
 'SPH': 'Sopu, Papua New Guinea: Sopu Airport',
 'SPI': 'Springfield, IL: Abraham Lincoln Capital',
 'SPK': 'Sapporo, Japan: Chitose AB',
 'SPM': 'Spangdahlem, Germany: Spangdahlem AB',
 'SPN': 'Saipan, TT: Francisco C. Ada Saipan International',
 'SPA': 'Spartanburg, SC: Spartanburg Downtown Memorial',
 'SPB': 'Charlotte Amalie, VI: Charlotte Amalie Harbor Seaplane Base',
 'SPC': 'Santa Cruz de la Palma, Spain: La Palma',
 'SPD': 'Saidpur, Bangladesh: Saidpur Airport',
 'SPE': 'Sepulot, Malaysia: Sepulot Airport',
 'SPF': 'Spearfish, SD: Black Hills Clyde Ice Field',
 'SPG': 'Tampa, F

In [56]:

# Lookup the airport name for all the airport codes
airportAvgDelay.map(lambda x: (airportLookup[x[0]],x[1])).take(10)

[('New York, NY: John F. Kennedy International', 11.334869814376479),
 ('Greer, SC: Greenville-Spartanburg International', 5.276119402985074),
 ('Flint, MI: Bishop International', 3.945945945945946),
 ('Rochester, NY: Greater Rochester International', 2.9452991452991455),
 ('Sitka, AK: Sitka Rocky Gutierrez', 0.06521739130434782),
 ('Miami, FL: Miami International', 15.01939832319579),
 ('Boston, MA: Logan International', 7.132262323943662),
 ('Oakland, CA: Metropolitan Oakland International', 8.376847290640393),
 ('Jacksonville/Camp Lejeune, NC: Albert J Ellis', 9.377777777777778),
 ('Little Rock, AR: Bill and Hillary Clinton Nat Adams Field',
  3.8794788273615635)]

In [57]:
# Broadcast Variable
airportBC=sc.broadcast(airportLookup)

In [1]:
airportAvgDelay.map(lambda x: (airportBC.value[x[0]],x[1])).sortBy(lambda x:-x[1]).take(10)

NameError: name 'airportAvgDelay' is not defined